# Impedance Calibration Test Run

**Overview:**
1. Network Preparation
1. Import Train and Test Sets
2. Specify Calibration Parameters
    - Link Impedance Function
    - Turn Impedance Function
    - Objective/Loss Function
        - First Preference Recovery
        - Exact Overlap
        - Buffer Overlap (in development)
        - Frechet Distance/Area (in development)
3. Run Calibration
    - Particle Swarm Optimization (constrained & non-probabilistic)
    - Maximum likelihood estimation (unconstrained & probabilistic, in development)
4. Export and Run Post Calibration


In [101]:
from pathlib import Path
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
import networkx as nx
from stochopy.optimize import minimize
import stochastic_optimization
from tqdm import tqdm
import similaritymeasures
import random

from shapely.ops import LineString, MultiLineString

import sys
sys.path.insert(0,str(Path.cwd().parent))
from network.src import modeling_turns
import speedfactor

In [102]:
import json
config = json.load((Path.cwd().parent / 'config.json').open('rb'))
calibration_fp = Path(config['project_directory']) / 'Calibration'
cycleatl_fp = Path(config['project_directory']) / 'CycleAtlanta'
matching_fp = Path(config['project_directory']) / 'Map_Matching'
network_fp = Path(config['project_directory']) / 'Network'
if calibration_fp.exists() == False:
    calibration_fp.mkdir()

# Network Preparation
- Determine which links should be included for routing
- Create directed edge dataframe
- Format edge and turn attribute variables
- Prepare dictionaries for quick edge attribute access

In [103]:
turns = pd.read_parquet(network_fp/'turns_df.parquet')
links = gpd.read_file(network_fp/'final_network.gpkg',layer='edges')


In [104]:
#dicts for referencing certain link attributes quickly
length_dict = dict(zip(links['linkid'],links['length_ft'])) # need this for loss function
geo_dict = dict(zip(links['linkid'],links['geometry']))

## Define which links are permitted for routing

In [105]:
#remove these types of links from routing
link_types_allowed = ['bike','pedestrian','road']
links['link_type'].unique()
links = links[links['link_type'].isin(link_types_allowed)]

highway_dict = dict(zip(links['linkid'],links['link_type']))
turns['source_link_type'] = turns['source_linkid'].map(highway_dict)
turns['target_link_type'] = turns['target_linkid'].map(highway_dict)
del highway_dict
turns = turns[turns['source_link_type'].isin(link_types_allowed) & turns['target_link_type'].isin(link_types_allowed)]

## Format link attributes

In [106]:
#Major/minor road classification to create high traffic stress variable
major_road = ['primary','secondary']
major_road = major_road + [item + '_link' for item in major_road]
minor_road = ['tertiary','unclassified','residential','service','trunk','living_street']
major_road = major_road + [item + '_link' for item in minor_road]
links.loc[links['highway'].isin(major_road),'link_type_new'] = 'major_road'
links.loc[links['highway'].isin(minor_road),'link_type_new'] = 'minor_road'
links.loc[links['link_type_new'].isna(),'link_type_new'] = links.loc[links['link_type_new'].isna(),'link_type']

links['high_traffic_stress'] = links['link_type_new'] == 'major_road'
links['motorized'] = links['link_type_new'].isin(['major_road','minor_road'])

# #Format variables (in progress)
# #HERE variables have error because of the conflation process
# above_30 = links['speedlimit_range_mph'].isin(['31-40 MPH','41-54 MPH','55-64 MPH'])
# more_than_1_lpd = links['lanes_per_direction'].isin(['2-3','> 4'])
# no_bike_infra = links['bike_facility_type'].isna()
# links['NACTO'] = 1
# links.loc[(above_30 | more_than_1_lpd) & no_bike_infra,'NACTO'] = 0
# links_geo = links['linkid'].map(geo_dict)
# links.reset_index(drop=True,inplace=True)
# links = gpd.GeoDataFrame(links,geometry=links_geo,crs='epsg:2240')
# links[links['NACTO']==0].explore()

## Format turn attributes
Only count left and right turns if going from one road to another road.

In [107]:
#sets turns that are not from road to road to None, effectively ignoring them
turns.loc[(turns['source_link_type']!='road') & (turns['target_link_type']!='road'),'turn_type'] = None

#create boolean turn type columns
turns['left'] = turns['turn_type'] == 'left'
turns['right'] = turns['turn_type'] == 'right'
turns['straight'] = turns['turn_type'] == 'straight'

#unsignalized straight/left turn where crossing street is a major road
highway_dict = dict(zip(links['linkid'],links['link_type_new']))
turns['source_link_type'] = turns['source_linkid'].map(highway_dict)
turns['target_link_type'] = turns['target_linkid'].map(highway_dict)
del highway_dict
turns['unsig_major_road_crossing'] = (turns['signalized']==False) & (turns['target_link_type']=='major_road') & (turns['source_link_type']=='minor_road')

#((turns['left']==True) | (turns['straight']==True))

In [108]:
turns['unsig_major_road_crossing'].value_counts()

False    240924
True       6808
Name: unsig_major_road_crossing, dtype: int64

In [109]:
#it looks like there are fewer signals than expected
turns['signalized'].value_counts()

False    236026
True      11706
Name: signalized, dtype: int64

## Import directed network

In [110]:
directed_links = pd.read_parquet(network_fp/'directed_edges.parquet')

#merge with links
link_cols_drop = ['facility_fwd','facility_rev','reverse_geometry','ascent_m', 'ascent_grade_%', 'descent_m', 'descent_grade_%']
links.drop(columns=link_cols_drop,inplace=True)
directed_cols_to_add = ['linkid','reverse_link','facility_fwd','ascent_m','ascent_grade_%']
links = pd.merge(links,directed_links[directed_cols_to_add])
del directed_links

# Remove wrongway
oneway_dict = dict(zip(links['linkid'],links['oneway']))
turns['source_oneway'] = turns['source_linkid'].map(oneway_dict)
turns['target_oneway'] = turns['target_linkid'].map(oneway_dict)
del oneway_dict
source_wrongway = ((turns['source_oneway'] == True) & (turns['source_reverse_link'] == True)) == False
target_wrongway = ((turns['target_oneway'] == True) & (turns['target_reverse_link'] == True)) == False
turns = turns[source_wrongway & target_wrongway]

#remove wrongway links
#TODO did we remove these in the export network step too?
links = links.loc[((links['oneway']==True) & (links['reverse_link']==True)) == False]#,'reverse_link'].value_counts()

## Format directed link attributes

In [111]:
#fill in missing NAs for grade
links['ascent_grade_%'] = links['ascent_grade_%'].fillna(0)
#TODO remove negative values from grade %
links['ascent_grade_%'] = links['ascent_grade_%'].abs()
#set any values above X to blank

In [112]:
#links['ascent_grade_%'].describe()
links['above_4'] = links['ascent_grade_%'] > 4

In [113]:
links['facility_fwd'].unique()

array([None, 'bike lane', 'multi use path', 'sharrow',
       'buffered bike lane', 'no facility', 'cycletrack'], dtype=object)

In [114]:
links['major_road_w_class_2'] = links['facility_fwd'].isin(['bike lane','buffered bike lane']) & (links['link_type_new'] == 'major road')
links['minor_road_w_class_2'] = links['facility_fwd'].isin(['bike lane','buffered bike lane']) & (links['link_type_new'] == 'minor road')
links['major_road_no_facil'] = (links['major_road_w_class_2'] == False) & (links['link_type_new'] == 'major road')
links['minor_road_no_facil'] = (links['minor_road_w_class_2'] == False) & (links['link_type_new'] == 'minor road')

In [115]:
#mixed traffic, no bike lanes
links['mixed_traffic_no_facil'] = links['motorized'] & (links['facility_fwd'].isin(['bike lane', 'multi use path', 'sharrow','buffered bike lane','cycletrack']) == False)
links['mixed_traffic_w_facil'] = links['motorized'] & (links['facility_fwd'].isin(['bike lane', 'multi use path', 'sharrow','buffered bike lane','cycletrack']) == True)

In [116]:
#add elevation adjusted travel times based on assumed speed on flat ground
speedfactor.calculate_adjusted_speed(links,9)

## Test Columns

In [117]:
# if you use these, you'll just get the shortest path back everytime
links['test_column'] = 0
turns['test_column'] = 0

# Form the turn graph needed for routing from the turns dataframe

In [118]:
turn_G = modeling_turns.make_turn_graph(turns)

In [119]:
#export calibration network
with (calibration_fp/"calibration_network.pkl").open('wb') as fh:
    pickle.dump((links,turns),fh)

# Specify Link Impedance Functions

In [120]:
links['mixed_traffic_no_facil'].value_counts()

True     114671
False     33078
Name: mixed_traffic_no_facil, dtype: int64

In [121]:
links['mixed_traffic_w_facil'].value_counts()

False    141389
True       6360
Name: mixed_traffic_w_facil, dtype: int64

In [122]:
#have position of beta next to name of variable
#NOTE: keys must be in the currect order used
betas_links = {
    0 : 'mixed_traffic_no_facil',
    1 : 'mixed_traffic_w_facil',
    #0 : 'major_road_w_class_2',
    # 1 : 'minor_road_w_class_2',
    # 2 : 'major_road_no_facil',
    # 3 : 'minor_road_no_facil',
    2 : 'above_4'
    #1 : 'motorized'
    #1 : 'ascent_grade_%'
} 

betas_turns = {
    3 : 'unsig_major_road_crossing'
    #1 : 'left',
    #2 : 'right',
    #3 : 'signalized'
}


In [123]:

'''
Currently works with binary and numeric variables. Categorical data will have to be
cast into a different format for now.

Link impedance is weighted by the length of the link, turns are just the impedance associated
'''

#customize this function to change impedance formula
#TODO streamline process of trying out new impedance functions
def link_impedance_function(betas,beta_links,links,base_impedance_col):
    #prevent mutating the original links gdf
    links = links.copy()
    
    multiplier = np.zeros(links.shape[0])
    
    if len(beta_links) > 0:
        #assumes that these effects are additive
        #TODO i think this can be done as a matrix product
        for key, item in beta_links.items():
            multiplier = multiplier + (betas[key] * links[item].values)
    
        links['link_cost'] = links[base_impedance_col] * (1 + multiplier) #removeing the + 1 for now

    else:
        links['link_cost'] = links[base_impedance_col]

    return links

def turn_impedance_function(betas,beta_turns,turns):
    #use beta coefficient to calculate turn cost
    # base_turn_cost = 30 # from Lowry et al 2016 DOI: http://dx.doi.org/10.1016/j.tra.2016.02.003
    # turn_costs = {
    #     'left': betas[1] * base_turn_cost,
    #     'right': betas[1] * base_turn_cost,
    #     'straight': betas[1] * base_turn_cost
    # }
    #turns['turn_cost'] = turns['turn_type'].map(turn_costs)

    turns = turns.copy()
    turns['turn_cost'] = 0

    if len(beta_turns) > 0:
        #instance impedance
        for key, item in beta_turns.items():
            turns['turn_cost'] = turns['turn_cost'] + (betas[key] * turns[item])

    #not sure if needed
    turns['turn_cost'] = turns['turn_cost'].astype(float)

    return turns

# Import Training Set

In [124]:
with (calibration_fp/'test_set.pkl').open('rb') as fh:
    test_set = pickle.load(fh)
with (calibration_fp/'train_set.pkl').open('rb') as fh:
    train_set = pickle.load(fh)

# import random
# random_trip = 1797#random.choice(list(train_set.keys()))
# train_set = {random_trip:train_set[random_trip]}

#match the ods to the network
train_ods = stochastic_optimization.match_results_to_ods(train_set)
test_ods = stochastic_optimization.match_results_to_ods(test_set)

In [125]:
# both_ods = list(set.union(set(train_ods),set(test_ods)))
# html = ""

# nodes = gpd.read_file(network_fp/'final_network.gpkg',layer='nodes')
# nodes.to_crs('epsg:4236',inplace=True)
# nodes['lon'] = nodes.geometry.x
# nodes['lat'] = nodes.geometry.y
# latlon = tuple(zip(nodes['lon'],nodes['lat']))
# nodes = dict(zip(nodes['N'],latlon))
# nodes.get(68196100,0)
# htmls = []
# for od in both_ods:
#     start = od[0]
#     end = od[1]
#     start_lonlat = nodes.get(start,0)
#     end_lonlat = nodes.get(end,0)
#     html = f"https://brouter.damsy.net/latest/#map=12/33.7522/-84.3892/standard&lonlats={start_lonlat[1]},{start_lonlat[0]};{end_lonlat[1]},{end_lonlat[0]}&profile=safety"
#     htmls.append(html)
# with (calibration_fp/"brouter_links.txt").open('w') as fh:
#     for html in htmls:
#         fh.write(f"{html}\n")

In [126]:
# gdf = train_set[random_trip]['matched_edges']
# gdf['geometry'] = gdf['linkid'].map(geo_dict)
# gdf = gpd.GeoDataFrame(gdf,crs=config['projected_crs_epsg'])
# gdf.explore()

# Calibration Settings

In [127]:
base_impedance_col = "travel_time_min"
loss_function = stochastic_optimization.jaccard_index
loss_function_kwargs = {'length_dict':length_dict}#,'overlap_threshold':0.80}

# loss_function = stochastic_optimization.buffer_overlap
# loss_function_kwargs = {'geo_dict':geo_dict,'buffer_ft':100,'standardize':True}

# link coefficients control the % increase in link travel time (units don't matter)
# turn coefficients control the amount of seconds added from the turn (units matter)
link_bounds = [[0, 2] for _ in range(0, len(betas_links))]
turn_bounds = [[0, 4] for _ in range(0, len(betas_turns))]
if (len(betas_links) > 0) & (len(betas_turns) > 0):
    bounds = np.vstack([link_bounds,turn_bounds])
elif (len(betas_links) > 0):
    bounds = link_bounds
elif (len(betas_turns) > 0):
    bounds = turn_bounds

In [128]:
past_betas = []
past_vals = []
args = (
    past_betas,
    past_vals,
    betas_links,betas_turns,
    train_ods,train_set,
    link_impedance_function,
    base_impedance_col,
    turn_impedance_function,
    links,turns,turn_G,
    loss_function,
    loss_function_kwargs,
    True #whether to print the results of each iteration
)

# Impedance Calibration

In [129]:
from importlib import reload
reload(stochastic_optimization)

start = time.time()
print(list(betas_links.values())+list(betas_turns.values())+['objective_function'])
x = minimize(stochastic_optimization.impedance_calibration, bounds, args=args, method='pso', options={'maxiter':15})
end = time.time()
print(f'Took {(end-start)/60/60:.2f} hours')

['mixed_traffic_no_facil', 'mixed_traffic_w_facil', 'above_4', 'unsig_major_road_crossing', 'objective_function']
[1.85, 0.81, 0.01, 1.7, -0.2524]
[0.24, 0.09, 1.3, 1.34, -0.3017]
[0.84, 0.6, 0.84, 2.93, -0.2836]
[1.05, 1.88, 0.41, 2.47, -0.2379]
[0.69, 1.25, 0.25, 0.95, -0.2405]
[1.43, 1.02, 1.03, 3.4, -0.2703]
[1.62, 0.47, 0.61, 3.71, -0.2629]
[1.24, 0.25, 1.62, 0.02, -0.2791]
[0.47, 1.41, 1.42, 0.41, -0.2325]
[0.06, 1.67, 1.86, 2.15, -0.2059]
[0.94, 0.24, 1.01, 1.37, -0.2909]
[0.24, 0.09, 1.3, 1.34, -0.3017]
[0.68, 0.08, 1.01, 2.86, -0.2958]
[0.04, 0.5, 0.9, 1.6, -0.2392]
[0.35, 0.11, 1.65, 1.34, -0.3111]
[1.35, 0.06, 1.33, 1.58, -0.2756]
[1.56, 0.07, 1.47, 1.37, -0.2664]
[1.09, 0.25, 1.65, 0.05, -0.2897]
[0.35, 0.45, 1.51, 1.75, -0.2846]
[0.31, 0.62, 1.55, 1.14, -0.2645]
[0.46, 0.27, 2.06, 1.08, -0.3028]
[0.38, 0.11, 1.31, 1.34, -0.3082]
[0.42, 0.34, 1.47, 1.7, -0.2997]
[0.67, 1.44, 1.67, 0.8, -0.2409]
[0.1, 0.72, 2.68, 1.63, -0.2176]
[0.61, 0.58, 1.55, 0.15, -0.2977]
[1.32, 0.17, 

In [130]:
# # MLE version to try out
# # Optimization to find the optimal beta coefficients
# result = minimize(loss_function, initial_beta, args=(G, observed_paths), method='BFGS')
# optimal_beta = result.x

# print(f"Optimal coefficients: {optimal_beta.round(2)}")

In [131]:
x

     fun: -0.31513428304183927
 message: 'maximum number of iterations is reached'
    nfev: 150
     nit: 15
  status: -1
 success: False
       x: array([0.46263004, 0.12362448, 1.53117212, 1.34265007])

In [132]:
#print('high stress,','ascent grade %,','left,','right,','signalized,','val')
print(list(betas_links.values())+list(betas_turns.values())+['objective_function'])
print(past_betas[np.array(past_vals).argmin()],np.array(past_vals).min().round(3))

['mixed_traffic_no_facil', 'mixed_traffic_w_facil', 'above_4', 'unsig_major_road_crossing', 'objective_function']
(0.46263003513032264, 0.12362448102583937, 1.5311721170426633, 1.3426500745160708) -0.315


In [133]:
#distribution of loss function values

In [134]:
combined_betas = {**betas_links, **betas_turns}
calibration_result = {}
#get the best betas
best_coefs = past_betas[np.array(past_vals).argmin()]
for key, item in combined_betas.items():
    calibration_result[item] = best_coefs[key]

calibration_result['loss'] = np.array(past_vals).min()
calibration_result['beta_links'] = betas_links
calibration_result['beta_turns'] = betas_turns
calibration_result

{'mixed_traffic_no_facil': 0.46263003513032264,
 'mixed_traffic_w_facil': 0.12362448102583937,
 'above_4': 1.5311721170426633,
 'unsig_major_road_crossing': 1.3426500745160708,
 'loss': -0.31513428304183927,
 'beta_links': {0: 'mixed_traffic_no_facil',
  1: 'mixed_traffic_w_facil',
  2: 'above_4'},
 'beta_turns': {3: 'unsig_major_road_crossing'}}

In [135]:
#export coefficents
if (calibration_fp/"calibration_results.pkl").exists():
    with (calibration_fp/"calibration_results.pkl").open('rb') as fh:
        calibration_results = pickle.load(fh)
else:
    calibration_results = []
calibration_results.append(calibration_result)

In [136]:
calibration_results

[{'high_traffic_stress': 0.0661610119497198,
  'left': 2.363333949708205,
  'right': 2.875007430851495,
  'signalized': 1.0893697547325298,
  'loss': -0.31058206517829084},
 {'class_1': 0.2308230358521835,
  'class_2': 1.2595525736224469,
  'class_3': 1.6777012261955218,
  'high_traffic_stress': 1.3901822221263547,
  'loss': -0.058962721570665844},
 {'major_road_w_class_2': 0.009352298764821576,
  'minor_road_w_class_2': 0.7192848670201979,
  'major_road_no_facil': 1.0242846535338108,
  'minor_road_no_facil': 1.4280555289480468,
  'above_4': 1.5152250035581623,
  'unsig_major_road_crossing': 0.012773275662048,
  'loss': -0.2903367458779148,
  'beta_links': {0: 'major_road_w_class_2',
   1: 'minor_road_w_class_2',
   2: 'major_road_no_facil',
   3: 'minor_road_no_facil',
   4: 'above_4'},
  'beta_turns': {5: 'unsig_major_road_crossing'}},
 {'mixed_traffic_no_facil': 0.46263003513032264,
  'mixed_traffic_w_facil': 0.12362448102583937,
  'above_4': 1.5311721170426633,
  'unsig_major_road_

In [137]:
#del calibration_results[-2]

In [138]:
with (calibration_fp/"calibration_results.pkl").open('wb') as fh:
        pickle.dump(calibration_results,fh)
calibration_results

[{'high_traffic_stress': 0.0661610119497198,
  'left': 2.363333949708205,
  'right': 2.875007430851495,
  'signalized': 1.0893697547325298,
  'loss': -0.31058206517829084},
 {'class_1': 0.2308230358521835,
  'class_2': 1.2595525736224469,
  'class_3': 1.6777012261955218,
  'high_traffic_stress': 1.3901822221263547,
  'loss': -0.058962721570665844},
 {'major_road_w_class_2': 0.009352298764821576,
  'minor_road_w_class_2': 0.7192848670201979,
  'major_road_no_facil': 1.0242846535338108,
  'minor_road_no_facil': 1.4280555289480468,
  'above_4': 1.5152250035581623,
  'unsig_major_road_crossing': 0.012773275662048,
  'loss': -0.2903367458779148,
  'beta_links': {0: 'major_road_w_class_2',
   1: 'minor_road_w_class_2',
   2: 'major_road_no_facil',
   3: 'minor_road_no_facil',
   4: 'above_4'},
  'beta_turns': {5: 'unsig_major_road_crossing'}},
 {'mixed_traffic_no_facil': 0.46263003513032264,
  'mixed_traffic_w_facil': 0.12362448102583937,
  'above_4': 1.5311721170426633,
  'unsig_major_road_

Create GIFs

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import imageio
from io import BytesIO

# Function to plot a GeoSeries and save the plot
def plot_geoseries(geoseries,other_geoseries,i,past_val):
    fig, ax = plt.subplots(figsize=(20, 20))
    #cx.add_basemap(ax)
    other_geoseries.plot(ax=ax,color='blue',style_kwds={'linewidth':2})
    geoseries.plot(ax=ax,color='red')
    ax.set_title(f"Iter:{i} Overlap Function:{past_val}")
    ax.set_axis_off()
    img_bytes = BytesIO()
    plt.savefig(img_bytes, format='png', bbox_inches='tight')
    plt.close()
    return img_bytes.getvalue()

In [ ]:
# num_trips = 10

# for z in range(0,num_trips):

#     #choose a random tripid
#     tripid = random.choice(list(train_set.keys()))
#     start_node = train_set[tripid]['start_node']
#     end_node = train_set[tripid]['end_node']

#     matched_edges = train_set[tripid]['matched_edges']
#     matched_edges = np.array(matched_edges)
#     matched_line = MultiLineString([geo_dict[linkid] for linkid, reverse_link in matched_edges])
#     matched_line = gpd.GeoSeries(matched_line,crs='epsg:2240')
#     matched_line = matched_line.to_crs('epsg:4326')

#     modeled_lines = []

#     for betas in past_betas:
#         #update network with the correct impedances
#         stochastic_optimization.impedance_update(betas,betas_links,betas_turns,
#                                 link_impedance_function,
#                                 turn_impedance_function,
#                                 links,turns,turn_G)
#         #find shortest path
#         modeled_edges = stochastic_optimization.impedance_path(turns,turn_G,start_node,end_node)['edge_list']
#         modeled_line = MultiLineString([geo_dict[linkid] for linkid, reverse_link in modeled_edges])
#         modeled_line = gpd.GeoSeries(modeled_line,crs='epsg:2240')
#         modeled_line = modeled_line.to_crs('epsg:4326')
#         modeled_lines.append(modeled_line)

#     # List of GeoSeries (Replace this with your own GeoSeries list)
#     geoseries_list = modeled_lines

#     # Loop through the list of GeoSeries, plot each one, and save the plot
#     images = []
#     for i, geoseries in enumerate(geoseries_list):
#         past_val = past_vals[i]
#         image_bytes = plot_geoseries(geoseries,matched_line,i,past_val)
#         images.append(imageio.imread(BytesIO(image_bytes)))

#     # Path for saving the GIF
#     gif_path = f"animations/stress_animation_{z}.gif"

#     # Save the images as a GIF
#     imageio.mimsave(Path.cwd()/gif_path, images, format='gif', duration=2)


# Validation

In [ ]:
from importlib import reload
reload(stochastic_optimization)

with (calibration_fp/"calibration_results.pkl").open('rb') as fh:
    calibration_results = pickle.load(fh)

In [ ]:
# #link_impedance_col = "adj_travel_time_min"
# stochastic_optimization.back_to_base_impedance(base_impedance_col,links,turns,turn_G)

# #update impedances
# betas = past_betas[np.array(past_vals).argmin()]#x.x
# print(betas)
# stochastic_optimization.impedance_update(betas,betas_links,betas_turns,
#                           link_impedance_function,
#                           base_impedance_col,
#                           turn_impedance_function,
#                           links,turns,turn_G)

# #find shortest path
# results_dict = {(start_node,end_node):stochastic_optimization.impedance_path(turns,turn_G,start_node,end_node) for start_node, end_node in test_ods}

# #calulate objective function
# val_to_minimize = loss_function(test_set,results_dict,**loss_function_kwargs)
# val_to_minimize.mean().round(2)

In [ ]:
#link_impedance_col = "adj_travel_time_min"
base_impedance_col = "travel_time_min"
stochastic_optimization.back_to_base_impedance(base_impedance_col,links,turns,turn_G)

#update impedances
betas = past_betas[np.array(past_vals).argmin()]#x.x
print(betas)
stochastic_optimization.impedance_update(betas,betas_links,betas_turns,
                          link_impedance_function,
                          base_impedance_col,
                          turn_impedance_function,
                          links,turns,turn_G)

#find shortest path
results_dict = {(start_node,end_node):stochastic_optimization.impedance_path(turns,turn_G,start_node,end_node) for start_node, end_node in train_ods}

#calulate objective function
val_to_minimize = loss_function(train_set,results_dict,**loss_function_kwargs)
val_to_minimize.mean().round(2)

## Visualize random trip

These did well

In [ ]:
fpr_results = stochastic_optimization.first_preference_recovery(train_set,results_dict,**{'length_dict':length_dict,'overlap_threshold':0.7})
fpr_results

In [ ]:
import random
tripid = random.choice(fpr_results)
tripid
#retrieve chosen path linkids and convert them to tuple
chosen = [tuple(row) for row in train_set[tripid]['matched_edges'].to_numpy()]
shortest = [tuple(row) for row in train_set[tripid]['shortest_edges'].to_numpy()]

#retrieve modeled path linkids
start_node = train_set[tripid]['origin_node']
end_node = train_set[tripid]['destination_node']
modeled_edges = results_dict[(start_node,end_node)]['edge_list']

#get geos (non-directional)
chosen_geo = [geo_dict[linkid[0]] for linkid in chosen]
shortest_geo = [geo_dict[linkid[0]] for linkid in shortest]
modeled_geo = [geo_dict[linkid[0]] for linkid in modeled_edges]

chosen_lines = gpd.GeoSeries(chosen_geo,crs='epsg:2240')
shortest_lines = gpd.GeoSeries(shortest_geo,crs='epsg:2240')
modeled_lines = gpd.GeoSeries(modeled_geo,crs='epsg:2240')

stochastic_optimization.visualize_three_no_legend(chosen_lines,shortest_lines,modeled_lines)

and these not so much

In [ ]:
import random
not_good = list(set(test_set.keys()) - set(fpr_results))

In [ ]:
tripid = random.choice(not_good)
tripid
#retrieve chosen path linkids and convert them to tuple
chosen = [tuple(row) for row in test_set[tripid]['matched_edges'].to_numpy()]
shortest = [tuple(row) for row in test_set[tripid]['shortest_edges'].to_numpy()]

#retrieve modeled path linkids
start_node = test_set[tripid]['origin_node']
end_node = test_set[tripid]['destination_node']
modeled_edges = results_dict[(start_node,end_node)]['edge_list']

#get geos (non-directional)
chosen_geo = [geo_dict[linkid[0]] for linkid in chosen]
shortest_geo = [geo_dict[linkid[0]] for linkid in shortest]
modeled_geo = [geo_dict[linkid[0]] for linkid in modeled_edges]

chosen_lines = gpd.GeoSeries(chosen_geo,crs='epsg:2240')
shortest_lines = gpd.GeoSeries(shortest_geo,crs='epsg:2240')
modeled_lines = gpd.GeoSeries(modeled_geo,crs='epsg:2240')

stochastic_optimization.visualize_three_no_legend(chosen_lines,shortest_lines,modeled_lines)